In [1]:
from sklearn.svm import SVR
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold,cross_validate

Cargo los datos y los divido en tres bloques para que no haya overfitting

In [2]:
#Carga de datos
data = pd.read_csv("acuario.csv")
print(data)
#Dividimos entre el goal que queremos sacar, que en este caso es el peso
y = data['Peso']
#El resto de los datos.
X = data.drop(columns=['Peso'])
#Dividimos las muestras dos veces para que el modelo trabaje con datos que no ha visto antes.
X_train_val, X_test, y_train_val, y_test = train_test_split(X,y,test_size=0.3)
X_train, X_val, y_train, y_val = train_test_split(X_train_val,y_train_val,test_size=0.3)

     Especie   Peso  Long_vert  Long_diag  Long_tras   Altura   Ancho
0          0  242.0       23.2       25.4       30.0  11.5200  4.0200
1          0  290.0       24.0       26.3       31.2  12.4800  4.3056
2          0  340.0       23.9       26.5       31.1  12.3778  4.6961
3          0  363.0       26.3       29.0       33.5  12.7300  4.4555
4          0  430.0       26.5       29.0       34.0  12.4440  5.1340
..       ...    ...        ...        ...        ...      ...     ...
154        4   12.2       11.5       12.2       13.4   2.0904  1.3936
155        4   13.4       11.7       12.4       13.5   2.4300  1.2690
156        4   12.2       12.1       13.0       13.8   2.2770  1.2558
157        4   19.7       13.2       14.3       15.2   2.8728  2.0672
158        4   19.9       13.8       15.0       16.2   2.9322  1.8792

[159 rows x 7 columns]


**PRUEBA CON RBF**

In [3]:

'''
kernel: 'linear', 'poly', 'rbf' o 'sigmoid'
degree: solo para kernel 'poly'
gamma: 'scale', 'auto' o float rbf sigmoid
'''
#Definimos valores bajos para las variables.
mejorScore = -10000000
nuevaC = 0
mejorEps = 0
mejorGamm = 0
#Hacemos bucles para encontrar la mejor configuración.
for c in [500,6000,7000,8000]:
    for eps in [1,5,10,15]:
        for gamm in [0.1,0.001,0.0001]:
            modelo = SVR(kernel="rbf",C=c, epsilon=eps,gamma=gamm)
            modelo.fit(X_train,y_train)#entrenamos
            score_actual= modelo.score(X_val,y_val)#sacamos el score actual
            if score_actual > mejorScore:
                mejorScore = score_actual
                nuevaC = c
                mejorEps = eps
                mejorGamm = gamm
                mejorModelo = modelo

mejorModelo.fit(X_train_val,y_train_val)#Entrenas con estos para que no sepa de los X_test para que no haga overfitting
print(f"Mejor score: {mejorScore}, mejor c: {nuevaC}, mejor eps: {mejorEps}, mejor gamma: {mejorGamm}")
print(f"Score de test: {mejorModelo.score(X_test,y_test)}")

#Forma corta y usando Grisearch.

print("\n ===Otra forma de hacerlo=== \n")
#Cargo los posibles parametros que se le van a pasar a el mdoelo.
parametros = {'C': [8000, 9000, 10000, 11000],'epsilon': [10, 15, 20, 25, 30],'gamma': [0.001, 0.01, 0.1, 1, 5, 10]}
#Cargo el modelo
modelo = SVR(kernel="rbf")
#Creo el modelo de grisearch con el modelo y os parametros que he creado antes.
gs = GridSearchCV(modelo, parametros, cv=5)
#Lo entreno con los datos de entreno
gs.fit(X_train_val, y_train_val)
#Mostrar los datos que se han generado con el entreno para poder comparar
print(f"Mejor score con los parametros de validación: {gs.best_score_}")
print(f"Mejores parámetros: {gs.best_params_}")
#Entrenamos con los datos de testa y mostramos lo datos para poder saber si el modelo y la configuración es buena.
score_test = gs.score(X_test, y_test)
print(f"Score final en el test: {score_test}")





Mejor score: 0.9873589796854976, mejor c: 8000, mejor eps: 1, mejor gamma: 0.001
Score de test: 0.9736697235320003

 ===Otra forma de hacerlo=== 

Mejor score con los parametros de validación: 0.9805494386074415
Mejores parámetros: {'C': 11000, 'epsilon': 10, 'gamma': 0.001}
Score final en el test: 0.9723332175437004


Vemos que con la configuración que hemos hecho, sale un score muy bueno, el cual está tanto con lo que aprende como lo que testea a mas de 95, lo que hace que sea muy fiable.

**PRUEBA CON LINEAR**

In [4]:

'''
kernel: 'linear', 'poly', 'rbf' o 'sigmoid'
degree: solo para kernel 'poly'
gamma: 'scale', 'auto' o float rbf sigmoid
'''
#Definimos valores bajos para las variables.
mejorScore = -10000000
nuevaC = 0
mejorEps = 0
#Hacemos bucles para encontrar la mejor configuración.
for c in [8000,9000,10000]:
    for eps in [15,20,25,30]:
            modelo = SVR(kernel="linear",C=c, epsilon=eps)
            modelo.fit(X_train,y_train)#entrenamos
            score_actual= modelo.score(X_val,y_val)#sacamos el score actual
            if score_actual > mejorScore:
                mejorScore = score_actual
                nuevaC = c
                mejorEps = eps
                mejorModelo = modelo

mejorModelo.fit(X_train_val,y_train_val)#Entrenas con estos para que no sepa de los X_test para que no haga overfitting
print(f"Mejor score: {mejorScore}, mejor c: {nuevaC}, mejor eps: {mejorEps}")
print(f"Score de test: {mejorModelo.score(X_test,y_test)}")

#Forma corta y usando Grisearch.

print("\n ===Otra forma de hacerlo=== \n")
#Cargo los posibles parametros que se le van a pasar a el mdoelo.
parametros = {'C': [8000, 9000, 10000, 11000],'epsilon': [10, 15, 20, 25, 30]}
#Cargo el modelo
modelo = SVR(kernel="rbf")
#Creo el modelo de grisearch con el modelo y os parametros que he creado antes.
gs = GridSearchCV(modelo, parametros, cv=5)
#Lo entreno con los datos de entreno
gs.fit(X_train_val, y_train_val)
#Mostrar los datos que se han generado con el entreno para poder comparar
print(f"Mejor score con los parametros de validación: {gs.best_score_}")
print(f"Mejores parámetros: {gs.best_params_}")
#Entrenamos con los datos de testa y mostramos lo datos para poder saber si el modelo y la configuración es buena.
score_test = gs.score(X_test, y_test)
print(f"Score final en el test: {score_test}")



Mejor score: 0.8201257227181811, mejor c: 8000, mejor eps: 30
Score de test: 0.8616258200329805

 ===Otra forma de hacerlo=== 

Mejor score con los parametros de validación: 0.9800085584256722
Mejores parámetros: {'C': 11000, 'epsilon': 15}
Score final en el test: 0.9680458368296099


Como podemos ver en los datos, este modelo sigue siendo bueno, pero no tanto como el anterior.
No se exactamente porque, pero con los datos con GridSearch salen bastante mejor datos.

**PRUEBA CON MODELO POLINÓMICO**

In [5]:

'''
kernel: 'linear', 'poly', 'rbf' o 'sigmoid'
degree: solo para kernel 'poly'
gamma: 'scale', 'auto' o float rbf sigmoid
'''
#Definimos valores bajos para las variables.
mejorScore = -10000000
nuevaC = 0
mejorEps = 0
mejorDegree = 0
#Hacemos bucles para encontrar la mejor configuración.
for c in [500,6000,7000,8000]:
    for eps in [1,5,10,15]:
        for degr in [1,2,3,4]:
            modelo = SVR(kernel="poly",C=c,degree=degr,epsilon=eps)
            modelo.fit(X_train,y_train)#entrenamos
            score_actual= modelo.score(X_val,y_val)#sacamos el score actual
            if score_actual > mejorScore:
                mejorScore = score_actual
                nuevaC = c
                mejorEps = eps
                mejorDegree = degr
                mejorModelo = modelo

mejorModelo.fit(X_train_val,y_train_val)#Entrenas con estos para que no sepa de los X_test para que no haga overfitting
print(f"Mejor score: {mejorScore}, mejor c: {nuevaC}, mejor eps: {mejorEps}, mejor grado: {mejorDegree}")
print(f"Score de test: {mejorModelo.score(X_test,y_test)}")
#Forma corta y usando Grisearch.

print("\n ===Otra forma de hacerlo=== \n")
#Cargo los posibles parametros que se le van a pasar a el mdoelo.
parametros = {'C': [500,6000,7000,8000],'epsilon': [10, 15, 20, 25, 30],'degree':[1,2,3,4]}
#Cargo el modelo
modelo = SVR(kernel="poly")
#Creo el modelo de grisearch con el modelo y os parametros que he creado antes.
gs = GridSearchCV(modelo, parametros, cv=5)
#Lo entreno con los datos de entreno
gs.fit(X_train_val, y_train_val)
#Mostrar los datos que se han generado con el entreno para poder comparar
print(f"Mejor score con los parametros de validación: {gs.best_score_}")
print(f"Mejores parámetros: {gs.best_params_}")
#Entrenamos con los datos de testa y mostramos lo datos para poder saber si el modelo y la configuración es buena.
score_test = gs.score(X_test, y_test)
print(f"Score final en el test: {score_test}")





Mejor score: 0.9564644979486272, mejor c: 6000, mejor eps: 5, mejor grado: 4
Score de test: 0.8770915293301781

 ===Otra forma de hacerlo=== 

Mejor score con los parametros de validación: 0.9557736727528325
Mejores parámetros: {'C': 500, 'degree': 3, 'epsilon': 10}
Score final en el test: 0.9795966459481558


Como podemos ver este modelo también es increiblemente eficiente ya que tenemos un score de 0.96, pero que sigue siendo menos que rbf, pero este modelo también sería perfectamente válido.

In [6]:

'''
kernel: 'linear', 'poly', 'rbf' o 'sigmoid'
degree: solo para kernel 'poly'
gamma: 'scale', 'auto' o float rbf sigmoid
'''
#Definimos valores bajos para las variables.
mejorScore = -10000000
nuevaC = 0
mejorEps = 0
mejorGamm = 0
#Hacemos bucles para encontrar la mejor configuración.
for c in [600,700,800,900]:
    for eps in [0.01,0.1,1,5,10]:
        for gamm in [0.1,0.001,0.0001]:
            modelo = SVR(kernel="sigmoid",C=c, epsilon=eps,gamma=gamm)
            modelo.fit(X_train,y_train)#entrenamos
            score_actual= modelo.score(X_val,y_val)#sacamos el score actual
            if score_actual > mejorScore:
                mejorScore = score_actual
                nuevaC = c
                mejorEps = eps
                mejorGamm = gamm
                mejorModelo = modelo

mejorModelo.fit(X_train_val,y_train_val)#Entrenas con estos para que no sepa de los X_test para que no haga overfitting
print(f"Mejor score: {mejorScore}, mejor c: {nuevaC}, mejor eps: {mejorEps}, mejor gamma: {mejorGamm}")
print(f"Score de test: {mejorModelo.score(X_test,y_test)}")

print("\n ===Otra forma de hacerlo=== \n")
#Cargo los posibles parametros que se le van a pasar a el mdoelo.
parametros = {'C': [600,700,800,900],'epsilon': [0.01,0.1,1,5,10],'gamma':[0.1,0.001,0.0001]}
#Cargo el modelo
modelo = SVR(kernel="sigmoid")
#Creo el modelo de grisearch con el modelo y os parametros que he creado antes.
gs = GridSearchCV(modelo, parametros, cv=5)
#Lo entreno con los datos de entreno
gs.fit(X_train_val, y_train_val)
#Mostrar los datos que se han generado con el entreno para poder comparar
print(f"Mejor score con los parametros de validación: {gs.best_score_}")
print(f"Mejores parámetros: {gs.best_params_}")
#Entrenamos con los datos de testa y mostramos lo datos para poder saber si el modelo y la configuración es buena.
score_test = gs.score(X_test, y_test)
print(f"Score final en el test: {score_test}")


Mejor score: 0.584436455157157, mejor c: 800, mejor eps: 10, mejor gamma: 0.0001
Score de test: 0.5822932905422253

 ===Otra forma de hacerlo=== 

Mejor score con los parametros de validación: 0.6710733344123718
Mejores parámetros: {'C': 600, 'epsilon': 0.01, 'gamma': 0.0001}
Score final en el test: 0.6305503073270086


Como podemos ver este modelo no es tan bueno como lo antesriores, ya que tiene un score bastante bajo en los test.

Al comparar todos los modelos, se puede ver claramente, que tanto el rbf y el polinómico, ya que superan el .95, con esto podemos decir que son muy buenos modelos para este dataset.